### 처리
1. 파라미터 설정
1. 매수 후보 종목 선별
1. 포트폴리오 조회 및 잔액갱신
1. 당일 스코아 계산
2. 매도 종목 검색 
4. 매수 종목 선정
1. (날짜 다음날 전환)
1. 매도처리
5. 매수처리


In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sqlite3
import csv
import random

%matplotlib notebook

In [52]:
# 전일영업일자 계산
def getBeforeWorkingDate(today):
    sql = "SELECT date FROM holiday_calendar WHERE date < {} AND holiday = 0".format(today)
    return pd.read_sql(sql, con, index_col=None).max()['date']

# 
def checkHoliday(day):
    sql = "SELECT holiday FROM holiday_calendar WHERE date = {}".format(day)
    return pd.read_sql(sql, con, index_col=None)['holiday'][0]

In [53]:
# *****************************
# 파라미터 설정
# *****************************
# 시작날짜
today = "20160105"
# 종료날짜
END_DAY = "20190120"
# 선별거래종목수
CANDIDATE_STOCK_CNT = 10
# 매수/매도 거래량
AMOUNT_BUY = 10
AMOUNT_SELL = 10
# 거래 주기 일자
CYCLE_DAY = 1
# 기준이동평균선
MOVING_AVERAGE1 = 'days5MovAvr'
MOVING_AVERAGE2 = 'days20MovAvr'

# directory
WORKING_DIR = "C:/Users/SDS/Documents/CybosPlus"
# db file
dbDir = 'C:/Users/SDS/Documents/CybosPlus/sqlite3'

In [54]:
# DBCP
con = sqlite3.connect(dbDir+"/MondayLab_init.db")
sqlite3.Connection
cursor = con.cursor()

### Loop 시작점

In [55]:
while today < END_DAY:
    

    # 날짜 설정
    beforeDay = getBeforeWorkingDate(today)
    print("*************************************************")
    print("today:[{}], beforDay:[{}]".format(today,beforeDay))

    # *****************************
    # 매수 후보 종목 선별
    # *****************************

    # kospi 200 종목 검색
    dfKospi200 = pd.read_sql("SELECT * FROM kospi200", con, index_col=None)

    # kospi200 종목중에서 매수 후보 주식 선별
    i = 0
    j = 0
    candidate = []
    for stockCode in dfKospi200['stockCode']:
        # 
        sqlTdoay = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                    AND stockCode = '{}' \
                    AND date = {} \
                    AND {} > {}" \
                    .format(stockCode,today,MOVING_AVERAGE1,MOVING_AVERAGE2)
        sqlBeforeDay = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                    AND stockCode = '{}' \
                    AND date = {} \
                    AND {} <= {}" \
                    .format(stockCode,beforeDay,MOVING_AVERAGE1,MOVING_AVERAGE2)

        result1 = pd.read_sql(sqlTdoay, con, index_col=None)

        if len(result1) != 0:
            result2 = pd.read_sql(sqlBeforeDay, con, index_col=None)

            if len(result2) != 0:
                j += 1
                candidate.append(result2['stockCode'][0])

        i += 1
    #     print("[{}]번째 진행중, [{}]데이터 보유".format(i,j))

    print("candidate : [{}]".format(len(candidate)))

    # 매수 후보 종목 csv 파일생성
    fileName = "{}/candidate_{}.csv".format(WORKING_DIR, today)
    with open(fileName, 'w') as csvFile:
        writer = csv.writer(csvFile)
        writer.writerow(candidate)

    csvFile.close()

    # *****************************
    # 포트폴리오 조회 및 잔액갱신
    # *****************************
    sqlPortfolio = "SELECT seq, stockCode, amountBuy, priceBuy FROM Portfolio WHERE 1=1 \
                    AND trading = 1"
    portfolio = pd.read_sql(sqlPortfolio, con, index_col=None)

    print("portfolio 건수 : [{}] ".format(len(portfolio)))

    # 잔액 갱신
    for index, myStock in portfolio.iterrows():
        sqlFinalPrice = "select stockCode, finalPrice from origin_trade where 1=1 \
                        and stockCode = '{}' \
                        and date = {}" \
                        .format(myStock['stockCode'], today)
#         print("sqlFinalPrice:[{}]".format(sqlFinalPrice))
        result = pd.read_sql(sqlFinalPrice, con, index_col=None)

        # 당일거래내역 없는 경우
        if len(result) == 0:
            continue
        
        # update
        priceSell = result['finalPrice'][0]*AMOUNT_SELL
        balance = priceSell - myStock['priceBuy']
        sqlUpdate = "update portfolio set \
                    dateSell = {} \
                    , amountSell = {} \
                    , priceSell = {} \
                    , balance = {} \
                    where 1=1 \
                      and seq = {}" \
                    .format(today, AMOUNT_SELL, priceSell, balance, myStock['seq'])
    #     print("sqlUpdate:[{}]".format(sqlUpdate))
        cursor.execute(sqlUpdate)
    con.commit()

    # *****************************
    # 당일 스코아 계산
    # *****************************
    sqlScore = "select sum(priceBuy) as totalInvest, sum(priceSell) as toltalCurrVal from Portfolio"
    resultScore = pd.read_sql(sqlScore, con, index_col=None)
    if resultScore['totalInvest'][0] != None:
        totalInvest = resultScore['totalInvest'][0]
        toltalCurrVal = resultScore['toltalCurrVal'][0]
        score = round(float(toltalCurrVal) / float(totalInvest),5)

        sqlIstScore = "insert into Score values ({},{}, {},{})" \
                      .format(today, totalInvest, toltalCurrVal, score)
#         print("sqlIstScore:[{}]".format(sqlIstScore))
        cursor.execute(sqlIstScore)
        con.commit()    
    
    # *****************************
    # 매도 종목 검색
    # *****************************
    stockSell = []
    if len(portfolio) != 0:
        for index, stcokSell in portfolio.iterrows():
            sqlStockSell = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                            AND stockCode = '{}' \
                            AND date = {} \
                            AND {} < {}" \
                            .format(stcokSell['stockCode'], today,MOVING_AVERAGE1,MOVING_AVERAGE2)
    #         print("sqlStockSell:[{}]".format(sqlStockSell))
            sellStockCode = pd.read_sql(sqlStockSell, con, index_col=None)

            if len(sellStockCode) != 0:
                stockSell.append([stcokSell['seq'], sellStockCode['stockCode'][0], stcokSell['priceBuy']])

    print("stockSell 건수:[{}]".format(len(stockSell)))


    # *****************************
    # 매수 종목 선정
    # *****************************
    buyStockCount = CANDIDATE_STOCK_CNT - len(portfolio) + len(stockSell)
    print("buyStockCount : [{}]".format(buyStockCount))

    # list에서 random 추출
    buyStocks = 0
    if len(candidate) <= buyStockCount:
        buyStocks = candidate
    else:
        buyStocks = random.sample(candidate, buyStockCount)


    # *****************************
    # (날짜 다음날 전환)
    # *****************************
    from datetime import datetime  
    from datetime import timedelta 

    temp = today
    for i in range(1,100):
        nextDate = datetime.strptime(str(today), "%Y%m%d") + timedelta(days=i)
        checkDay = nextDate.strftime('%Y%m%d')
        if checkHoliday(checkDay) == 0:
            break
    today = checkDay
    print("chage today [{}] to [{}]".format(temp, today))


    # *****************************
    # 매도 처리
    # *****************************
    if len(stockSell) != 0:
        for sellStockCode in stockSell:
            # 시초가 거래
            sqlSell = "select startPrice from origin_trade where 1=1 \
                        AND stockCode = '{}' \
                        AND date = {}" \
                        .format(sellStockCode[1], today)
#             print("sqlSell:[{}]".format(sqlSell))
            startPrice = pd.read_sql(sqlSell, con, index_col=None)

            # 당일거래내역 없는 경우
            if len(startPrice) == 0:
                continue
            
            # Trading 매도 내역 생성
            stockPrice = startPrice['startPrice'][0]*AMOUNT_SELL
            sqlInstSellTrd = "insert into Trading values ({},{},'{}',{},{})" \
                            .format(today, 2, sellStockCode[1], AMOUNT_SELL, stockPrice)
    #         print("sqlInstSellTrd:[{}]".format(sqlInstSellTrd))
            cursor.execute(sqlInstSellTrd)

            # Portfolio 갱신
            balance = stockPrice - sellStockCode[2]
            seq = sellStockCode[0]
            sqlUpdate = "update portfolio set \
                        trading = 2 \
                        , dateSell = {} \
                        , amountSell = {} \
                        , priceSell = {} \
                        , balance = {} \
                        where 1=1 \
                          and seq = {}" \
                        .format(today, AMOUNT_SELL, stockPrice, balance, seq)
    #         print("sqlUpdate:[{}]".format(sqlUpdate))
            cursor.execute(sqlUpdate)

        con.commit()




    # *****************************
    # 매수처리
    # *****************************
    for stockBuyCode in buyStocks:
        # 시초가 거래
        sqlBuy = "select startPrice from origin_trade where 1=1 \
                  AND stockCode = '{}' \
                  AND date = {}" \
                  .format(stockBuyCode, today)
#         print("sqlBuy:[{}]".format(sqlBuy))
        startPrice = pd.read_sql(sqlBuy, con, index_col=None)

        # 당일 거래내역이 없는 경우
        if len(startPrice) == 0:
            continue
    
        stockPrice = startPrice['startPrice'][0] * AMOUNT_BUY
    
        # Trading 매수 내역 생성
        sqlInstBuyTrd = "insert into Trading values({},{},'{}',{},{})" \
                       .format(today, 1, stockBuyCode, AMOUNT_BUY, stockPrice)
        cursor.execute(sqlInstBuyTrd)

        # Portfolio 생성
        sqlSeq = "select max(seq) as maxSeq from Portfolio"
        seq = pd.read_sql(sqlSeq, con, index_col=None)['maxSeq'][0]
        if seq == None:
            seq = 0
        seq += 1
        trading = 1

        sqlInsert = "insert into Portfolio values({},{},'{}',{},{},{},{},{},{},{})" \
                    .format(seq, trading, stockBuyCode, today, AMOUNT_BUY, stockPrice,0,0,0,0)
        cursor.execute(sqlInsert)

    con.commit()
    


*************************************************
today:[20160105], beforDay:[20160104]
candidate : [6]
portfolio 건수 : [0] 
stockSell 건수:[0]
buyStockCount : [10]
chage today [20160105] to [20160106]
*************************************************
today:[20160106], beforDay:[20160105]
candidate : [6]
portfolio 건수 : [6] 
stockSell 건수:[0]
buyStockCount : [4]
chage today [20160106] to [20160107]
*************************************************
today:[20160107], beforDay:[20160106]
candidate : [1]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160107] to [20160108]
*************************************************
today:[20160108], beforDay:[20160107]
candidate : [2]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160108] to [20160111]
*************************************************
today:[20160111], beforDay:[20160108]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160111] to [20160112]
******

candidate : [10]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160308] to [20160309]
*************************************************
today:[20160309], beforDay:[20160308]
candidate : [6]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160309] to [20160310]
*************************************************
today:[20160310], beforDay:[20160309]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160310] to [20160311]
*************************************************
today:[20160311], beforDay:[20160310]
candidate : [6]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160311] to [20160314]
*************************************************
today:[20160314], beforDay:[20160311]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160314] to [20160315]
*************************************************
today:[20160315], beforDay:[20160314]
cand

candidate : [6]
portfolio 건수 : [7] 
stockSell 건수:[2]
buyStockCount : [5]
chage today [20160510] to [20160511]
*************************************************
today:[20160511], beforDay:[20160510]
candidate : [7]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20160511] to [20160512]
*************************************************
today:[20160512], beforDay:[20160511]
candidate : [5]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160512] to [20160513]
*************************************************
today:[20160513], beforDay:[20160512]
candidate : [0]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160513] to [20160516]
*************************************************
today:[20160516], beforDay:[20160513]
candidate : [10]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160516] to [20160517]
*************************************************
today:[20160517], beforDay:[20160516]
candi

candidate : [9]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160708] to [20160711]
*************************************************
today:[20160711], beforDay:[20160708]
candidate : [6]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160711] to [20160712]
*************************************************
today:[20160712], beforDay:[20160711]
candidate : [8]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160712] to [20160713]
*************************************************
today:[20160713], beforDay:[20160712]
candidate : [12]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20160713] to [20160714]
*************************************************
today:[20160714], beforDay:[20160713]
candidate : [12]
portfolio 건수 : [10] 
stockSell 건수:[3]
buyStockCount : [3]
chage today [20160714] to [20160715]
*************************************************
today:[20160715], beforDay:[20160714]
can

candidate : [6]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160907] to [20160908]
*************************************************
today:[20160908], beforDay:[20160907]
candidate : [11]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20160908] to [20160909]
*************************************************
today:[20160909], beforDay:[20160908]
candidate : [8]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20160909] to [20160912]
*************************************************
today:[20160912], beforDay:[20160909]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20160912] to [20160913]
*************************************************
today:[20160913], beforDay:[20160912]
candidate : [0]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20160913] to [20160919]
*************************************************
today:[20160919], beforDay:[20160913]
cand

...

candidate : [5]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20180502] to [20180503]
*************************************************
today:[20180503], beforDay:[20180502]
candidate : [7]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20180503] to [20180504]
*************************************************
today:[20180504], beforDay:[20180503]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20180504] to [20180508]
*************************************************
today:[20180508], beforDay:[20180504]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[2]
buyStockCount : [2]
chage today [20180508] to [20180509]
*************************************************
today:[20180509], beforDay:[20180508]
candidate : [4]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20180509] to [20180510]
*************************************************
today:[20180510], beforDay:[20180509]
candi

candidate : [0]
portfolio 건수 : [9] 
stockSell 건수:[1]
buyStockCount : [2]
chage today [20180705] to [20180706]
*************************************************
today:[20180706], beforDay:[20180705]
candidate : [2]
portfolio 건수 : [8] 
stockSell 건수:[1]
buyStockCount : [3]
chage today [20180706] to [20180709]
*************************************************
today:[20180709], beforDay:[20180706]
candidate : [7]
portfolio 건수 : [9] 
stockSell 건수:[0]
buyStockCount : [1]
chage today [20180709] to [20180710]
*************************************************
today:[20180710], beforDay:[20180709]
candidate : [9]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20180710] to [20180711]
*************************************************
today:[20180711], beforDay:[20180710]
candidate : [8]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20180711] to [20180712]
*************************************************
today:[20180712], beforDay:[20180711]
candidat

candidate : [3]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20180904] to [20180905]
*************************************************
today:[20180905], beforDay:[20180904]
candidate : [7]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20180905] to [20180906]
*************************************************
today:[20180906], beforDay:[20180905]
candidate : [3]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20180906] to [20180907]
*************************************************
today:[20180907], beforDay:[20180906]
candidate : [1]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20180907] to [20180910]
*************************************************
today:[20180910], beforDay:[20180907]
candidate : [12]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20180910] to [20180911]
*************************************************
today:[20180911], beforDay:[20180910]
cand

candidate : [19]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20181108] to [20181109]
*************************************************
today:[20181109], beforDay:[20181108]
candidate : [12]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20181109] to [20181112]
*************************************************
today:[20181112], beforDay:[20181109]
candidate : [20]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20181112] to [20181113]
*************************************************
today:[20181113], beforDay:[20181112]
candidate : [6]
portfolio 건수 : [10] 
stockSell 건수:[2]
buyStockCount : [2]
chage today [20181113] to [20181114]
*************************************************
today:[20181114], beforDay:[20181113]
candidate : [11]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20181114] to [20181115]
*************************************************
today:[20181115], beforDay:[20181114]
c

candidate : [9]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20190110] to [20190111]
*************************************************
today:[20190111], beforDay:[20190110]
candidate : [16]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20190111] to [20190114]
*************************************************
today:[20190114], beforDay:[20190111]
candidate : [8]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20190114] to [20190115]
*************************************************
today:[20190115], beforDay:[20190114]
candidate : [8]
portfolio 건수 : [10] 
stockSell 건수:[1]
buyStockCount : [1]
chage today [20190115] to [20190116]
*************************************************
today:[20190116], beforDay:[20190115]
candidate : [7]
portfolio 건수 : [10] 
stockSell 건수:[0]
buyStockCount : [0]
chage today [20190116] to [20190117]
*************************************************
today:[20190117], beforDay:[20190116]
cand

#### reset

In [50]:
truncateTabel = ['Score', 'Portfolio','Trading']
for table in truncateTabel:
    cursor.execute("delete from {}".format(table))
con.commit()